<a href="https://colab.research.google.com/github/kuonumber/crawler/blob/master/Day14_homework_Yahoo_movie_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
import requests
import re
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
# 先觀察一下目前上映中的電影數量
url = 'https://movies.yahoo.com.tw/movie_intheaters.html'
resp = requests.get(url)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
html = soup.find("div", attrs={'class':'release_box'})  # 尋找正在上映中的全部電影筆數，其所在的tag
print("正在上映中總共: ", html.p.string)

正在上映中總共:  共89筆，目前顯示1~10筆


In [14]:
# 接下來要爬取每一頁的電影介紹
# 先點選網頁最底部的下一頁，觀察其網址變化。然後我們把頁數當作變數傳入，用迴圈爬取。
max_page = 5
for page_number in range(1, max_page+1, 1):
    url = 'https://movies.yahoo.com.tw/movie_intheaters.html'
    payload = {'page':str(page_number)}
    resp = requests.get(url, params=payload)
    resp.encoding = 'utf-8'
    soup = BeautifulSoup(resp.text, 'lxml')

    # 把電影介紹擷取出來
    movie_list = soup.find("ul", attrs={"class":"release_list"}).find_all("li")
    for p in movie_list:
        print("----------------------------------------------------------------------")
        # 電影名稱
        movie_name = p.find("div", attrs={"class":"release_movie_name"})
        print("電影名稱：", movie_name.a.string)
        
        # 定位電影評價資訊欄位
        level_box = movie_name.find("dl", attrs={"class":"levelbox"})
        
        # 期待度
        expectation = level_box.findAll("div", attrs={"class":"leveltext"})[0]
        print("期待度：", expectation.span.string)
        
        # 滿意度
        satisfaction = level_box.findAll("div", attrs={"class":"leveltext"})[1]
        print("滿意度：", satisfaction.span["data-num"])  # 滿意度可以從"data-num"這個屬性擷取
        
        # 簡介
        movie_info = p.find("div", attrs={"class":"release_text"})
        print(movie_info.span.string)

----------------------------------------------------------------------
電影名稱： 
                  一罐子的美好人生
期待度： 88%
滿意度： 3.8

                  ★ 年輕的流浪，是一生的養分！花樣少女的壯遊冒險之旅即將展開！
★ 一生一次的壯遊，增添人生的無限可能
★《沒有耳朵的兔子》《最酷的一天》德國演技派男星馬提亞斯史維克福 暖心出演　　
 
20歲出頭的少女瑪琳正值青春年華，仍與父母同住的她，對於人生似乎沒有特別的想法或目標，她深陷在那些看似難以克服的日常限制之中，就算是跟她感情極好的哥哥也無法讓她提振精神。某日，一場悲劇意外發生，出於罪惡感和絕望感，瑪琳踏上了一段事先毫無準備的冒險旅程，身上只有八歐元的她，跳上了停泊漢堡的船隻前往南極洲。在這場壯遊之旅，瑪琳逐漸認識到這個世界其實充滿了許多超乎她想像的人事物，她的人生視野也有了意想不到的新發現：這場壯遊離家越遠，她就越接近真實的自己…
 
                
----------------------------------------------------------------------
電影名稱： 
                  馬拉薩尼亞32號陰宅
期待度： 91%
滿意度： 3.7

                  ★ 繼《厲陰宅》後，暑假壓軸恐怖懼獻
★ 《佈局》《屍物招領》製片破膽打造
★ 改編自西班牙史上最陰真實案件之一
★ 平均兩分鐘一次驚嚇，你敢挑戰嗎？
★ 住進夢寐以求的新家，卻讓他們噩夢不斷…
    
改編自70年代西班牙發生的真實駭人事件，一對夫妻帶著三個孩子與年邁的父親入住一間位於馬德里馬拉薩尼亞的公寓，然而這家人卻渾然不知，他們買的這間房子即將成為他們此生最大的噩夢……。
                
----------------------------------------------------------------------
電影名稱： 
                  紐約茶館
期待度： 75%
滿意度： 3.3

                  ★ 柏林影展競賽片 開幕片 Competition 

In [15]:
# 查看目前上映那些電影，並擷取出其ID資訊
url = 'https://movies.yahoo.com.tw/'
resp = requests.get(url)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
html = soup.find("select", attrs={'name':'movie_id'})
movie_item = html.find_all("option", attrs={'data-name':re.compile('.*')})

for p in movie_item:
    print("Movie: %s, ID: %s" % (p["data-name"], p["value"]))

Movie: 我是大哥大！！劇場版, ID: 10871
Movie: 花木蘭, ID: 8632
Movie: 馗降：粽邪2, ID: 10787
Movie: 八月三十一日，我在奧斯陸, ID: 10885
Movie: 美麗佳人歐蘭朵, ID: 10876
Movie: 蘇州河, ID: 10858
Movie: 紐約茶館, ID: 10856
Movie: 馬拉薩尼亞32號陰宅, ID: 10854
Movie: 一罐子的美好人生, ID: 10848
Movie: 變種人, ID: 10452
Movie: TENET天能, ID: 10433
Movie: 最後的情書, ID: 10880
Movie: 聖雅各的天空, ID: 10877
Movie: 看不見的證人, ID: 10873
Movie: 午夜0時的吻, ID: 10855
Movie: 初心, ID: 10837
Movie: 高盧英雄歷險記：魔法藥水, ID: 10834
Movie: 下一站，托斯卡尼, ID: 10817
Movie: 冬天的故事 經典數位修復, ID: 10739
Movie: 秋天的故事 經典數位修復, ID: 10738
Movie: 可不可以，你也剛好喜歡我, ID: 10473
Movie: 羊與鋼之森, ID: 8460
Movie: 棕櫚泉不思議, ID: 10864
Movie: 東京教父：4K數位修復版, ID: 10860
Movie: Fate/stay night [Heaven's Feel] III.春櫻之歌, ID: 10630
Movie: 劇場版 新幹線變形機器人—來自未來的神速ALFA-X, ID: 10823
Movie: 我的兒子是死刑犯, ID: 10782
Movie: 春天的故事 經典數位修復, ID: 10736
Movie: 逃出立法院, ID: 10535
Movie: 全面啟動10周年紀念版, ID: 10867
Movie: 天劫倒數, ID: 10850
Movie: 魔鬼對決, ID: 10842
Movie: 皮行者, ID: 10794
Movie: 地下社會, ID: 10840
Movie: 心靈咖啡館的驅魔師, ID: 10830
Movie: 6號鬼病床, ID: 10826
Movie: 哈囉少

In [16]:
movie_id = 10429


In [17]:
url = 'https://movies.yahoo.com.tw/api/v1/areas_by_movie_theater'
payload = {'movie_id':str(movie_id)}

# 模擬一個header
headers = {
    'authority': 'movies.yahoo.com.tw',
    'method': 'GET',
    'path': '/api/v1/areas_by_movie_theater?movie_id=' + str(movie_id),
    'scheme': 'https',
    'accept': 'application/json, text/javascript, */*; q=0.01',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'rxx=9s3x2fws06.1g16irnc&v=1; _ga=GA1.3.2056742944.1551651301; GUC=AQEBAQFczFpdm0IfmwSB&s=AQAAACoo4N5D&g=XMsVBw; BX=4hkdk1decm57t&b=3&s=mr; _ga=GA1.4.2056742944.1551651301; nexagesuid=82843256dd234e8e91aa73f2062f8218; browsed_movie=eyJpdiI6IlJXWWtiSWJaZlNGK2MxQnhscnVUYWc9PSIsInZhbHVlIjoiMXRhMmVHRXRIeUNjc1RBWDJzdGYwbnlIQURmWGsrcjJSMzhkbkcraDNJVUNIZEZsbzU3amlFcVZ1NzlmazJrTGpoMjVrbHk1YmpoRENXaHZTOUw1TmI2ZTZVWHdOejZQZm16RmVuMWlHTTJLaTZLVFZZVkFOMDlTd1wvSGltcytJIiwibWFjIjoiZWQ2ZjA4MmVjZmZlYjlmNjJmYmY2NGMyMDI0Njc0NWViYjVkOWE2NDg0N2RhODMxZjBjZDhiMmJhZTc2MDZhYiJ9; avi=eyJpdiI6Im1NeWFJRlVRWDR1endEcGRGUGJUbVE9PSIsInZhbHVlIjoickRpU3JuUytmcGl6cjF5OW0rNU9iZz09IiwibWFjIjoiY2VmY2NkNzZmM2NhNjY5YzlkOTcyNjE5OGEyMzU0NWYxOTdmMDRkMDY3OWNmMmZjOTMxYjc5MjI5N2Q5NGE5MiJ9; cmp=t=1559391030&j=0; _gid=GA1.4.779543841.1559391031; XSRF-TOKEN=eyJpdiI6IkhpS2hGcDRQaHlmWUJmaHdSS2Q2bHc9PSIsInZhbHVlIjoiOUVoNFk4OHI1UUZmUWRtYXhza0MyWjJSTlhlZ3RnT0VGeVJPN2JuczVRMGRFdWt2OUlsamVKeHRobFwvcHBGM0dhU3VyMXNGTHlsb2dVM2l0U1hpUGxBPT0iLCJtYWMiOiJkZWU4YzJhNjAxMTY3MzE4Y2ExNWIxYmE1ZjE1YWZlZTlhOTcyYjc4M2RlZGY4ZWNjZDYyMTA2NGYwZGViMzc2In0%3D; m_s=eyJpdiI6InpsZHZ2Tk1BZ0dxaHhETml1RjBnUXc9PSIsInZhbHVlIjoiSkNGeHUranRoXC85bDFiaDhySTJqNkJRcWdjWUxjeVRJSHVYZ1wvd2d4bWJZUTUrSHVDM0lUcW5KNHdETFZ4T1lieU81OUhzc1VoUXhZcWk0UDZSQXVFdz09IiwibWFjIjoiYmJkMDJkMDhlODIzMzcyMWY4M2NmYWNjNGVlOWRjMDIwZmVmNzAyMjE3Yzg3ZGY3ODBkZWEzZTI4MTI5ZWNmOSJ9; _gat=1; nexagesd=10',
    'dnt': '1',
    'mv-authorization': '21835b082e15b91a69b3851eec7b31b82ce82afb',
    'referer': 'https://movies.yahoo.com.tw/',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36',
    'x-requested-with': 'XMLHttpRequest',
}
    
resp = requests.get(url, params=payload, headers=headers)
#print(resp.json())  # 若有需要，列印出json原始碼

# 這裡回傳的格式是JSON格式的資料，要解析JSON擷取資料
for p in resp.json():
    print('放映地區: {0}, 代號(area_id): {1}'.format(p['title'], p['area_id']))

放映地區: 台北市, 代號(area_id): 28
放映地區: 新北市, 代號(area_id): 8
放映地區: 桃園, 代號(area_id): 16
放映地區: 新竹, 代號(area_id): 20
放映地區: 苗栗, 代號(area_id): 15
放映地區: 台中, 代號(area_id): 2
放映地區: 台南, 代號(area_id): 10
放映地區: 高雄, 代號(area_id): 17


In [32]:
area_id = 28


In [33]:
# 向網站發送請求
url = 'https://movies.yahoo.com.tw/movietime_result.html'
payload = {'movie_id':str(movie_id), 'area_id':str(area_id)}
resp = requests.get(url, params=payload)
resp.encoding = 'utf-8'

soup = BeautifulSoup(resp.text, 'lxml')
movie_date = soup.find_all("label", attrs={'for':re.compile("date_[\d]")})

# 列印播放日期
for date in movie_date:
    print("%s %s" % (date.p.string, date.h3.string))

九月 3
九月 4
九月 5
九月 6
九月 7


In [34]:
# 選定要觀看的日期
date = "2020-09-06"

In [35]:
# 向網站發送請求，獲取上映的電影院及時間資訊
url = "https://movies.yahoo.com.tw/ajax/pc/get_schedule_by_movie"
payload = {'movie_id':str(movie_id),
           'date':date,
           'area_id':str(area_id),
           'theater_id':'',
           'datetime':'',
           'movie_type_id':''}

resp = requests.get(url, params=payload)
#print(resp.json()['view'])  # 若有需要，列印出json原始碼

soup = BeautifulSoup(resp.json()['view'], 'lxml')
html = soup.find_all("ul", attrs={'data-theater_name':re.compile(".*")})

In [36]:
#   試著從上一步驟回傳的電影院資料中，擷取電影院名稱、影片放映類型以及時間表


In [37]:
soup.find_all("ul", attrs={'data-theater_name':re.compile(".*")})

[<ul class="area_time _c jq_area_time" data-theater_name="京站威秀影城" data-theater_schedules="https://movies.yahoo.com.tw/theater_result.html/id=118" data-theater_url="http://www.vscinemas.com.tw/" id="theater_id_118">
 <li class="adds">
 <a href="https://movies.yahoo.com.tw/theater_result.html/id=118">京站威秀影城</a>
 <span>02-25525511</span>
 </li>
 <li class="taps">
 <span class="tapR">數位</span> </li>
 <li class="time _c">
 <div class="input_picker jq_input_picker">
 <input class="gabtn" data-cinema="5f60dacd-45b3-102d-8d74-e400529c" data-date="1599321600" data-ezding="ezding" data-ga="['電影頁_時刻表', '電影頁_時刻表_場次', '巧虎電影：英雄遊樂園']" data-movie="fceff273cf574dd9bfbed73d06d49d7a" data-movie_date="09.06" data-movie_time="10:10" data-movie_title="巧虎電影：英雄遊樂園" data-movie_type="數位" data-session="6da16b75216c47b799c4cb39ee035110" data-time="1599358200" id="74071167" name="schedule_list" type="radio" value="2020-09-06 10:10:00"/>
 <label class="select" for="74071167">10:10</label>
 </div>
 </li>
 </ul>,
 <u

In [83]:
pd.concat([pd.DataFrame(i.select('.gabtn')[0].attrs for i in soup.select('ul.area_time._c.jq_area_time')])
# soup.select('ul')[0].select('.gabtn')[0]['data-movie_date']
# soup.select('ul')[0].select('.gabtn')[0]['data-date']
# soup.select('ul')[0].select('.gabtn')[0]['data-ga']
# soup.select('ul')[0].select('.gabtn')[0]['data-movie_type']
# soup.select('ul')[0].select('.gabtn')[0].attrs

,type,name,id,class,value,data-movie,data-cinema,data-session,data-date,data-time,data-movie_type,data-movie_title,data-movie_date,data-movie_time,data-ga,data-ezding
0,radio,schedule_list,74071167,gabtn,2020-09-06 10:10:00,fceff273cf574dd9bfbed73d06d49d7a,5f60dacd-45b3-102d-8d74-e400529c,6da16b75216c47b799c4cb39ee035110,1599321600,1599358200,數位,巧虎電影：英雄遊樂園,09.06,10:10,"['電影頁_時刻表', '電影頁_時刻表_場次', '巧虎電影：英雄遊樂園']",ezding
0,radio,schedule_list,74070218,gabtn,2020-09-06 10:10:00,426b606674c047e39557ac4b93995954,470e99e70b5611eaa91e06ba2bcebcfa,3035cd636cff40b0879497988f17a856,1599321600,1599358200,數位,巧虎電影：英雄遊樂園,09.06,10:10,"['電影頁_時刻表', '電影頁_時刻表_場次', '巧虎電影：英雄遊樂園']",ezding


In [78]:
soup.select('ul.area_time._c.jq_area_time')

[<ul class="area_time _c jq_area_time" data-theater_name="京站威秀影城" data-theater_schedules="https://movies.yahoo.com.tw/theater_result.html/id=118" data-theater_url="http://www.vscinemas.com.tw/" id="theater_id_118">
 <li class="adds">
 <a href="https://movies.yahoo.com.tw/theater_result.html/id=118">京站威秀影城</a>
 <span>02-25525511</span>
 </li>
 <li class="taps">
 <span class="tapR">數位</span> </li>
 <li class="time _c">
 <div class="input_picker jq_input_picker">
 <input class="gabtn" data-cinema="5f60dacd-45b3-102d-8d74-e400529c" data-date="1599321600" data-ezding="ezding" data-ga="['電影頁_時刻表', '電影頁_時刻表_場次', '巧虎電影：英雄遊樂園']" data-movie="fceff273cf574dd9bfbed73d06d49d7a" data-movie_date="09.06" data-movie_time="10:10" data-movie_title="巧虎電影：英雄遊樂園" data-movie_type="數位" data-session="6da16b75216c47b799c4cb39ee035110" data-time="1599358200" id="74071167" name="schedule_list" type="radio" value="2020-09-06 10:10:00"/>
 <label class="select" for="74071167">10:10</label>
 </div>
 </li>
 </ul>,
 <u